In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
import cv2
import random

In [ ]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.vgg import VGG16_BN_Weights
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
MODEL_PATH = "../scripts/from_scratch/model.pth"
num_classes = 39

In [ ]:
model_ft = models.vgg16_bn()
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
model_ft.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
target_layers = [model_ft.features[-1]]
model_ft.eval()

In [ ]:
classes_dict = {}
BASE_DIR = "../../dataset/Plant_leave_diseases_dataset_without_augmentation/"
for dir_name in os.listdir(BASE_DIR):
    dir_path = os.path.join(BASE_DIR, dir_name)
    # Check if it's a directory
    if os.path.isdir(dir_path):
        # Get all files in the directory
        all_files = os.listdir(dir_path)
        random_images = random.sample(all_files, 12)
        # Append the images to the dictionary with the directory name as the key
        classes_dict[dir_name] = []
        for image in random_images:
            classes_dict[dir_name].append(os.path.join(BASE_DIR, dir_path, image))

In [ ]:
for dir_name, images_list in classes_dict.items():
    # Create a figure for the current directory with 10 subplots (1 row x 10 columns)
    fig, axes = plt.subplots(4, 6, figsize=(20, 15))
    fig.suptitle(dir_name)  # Set the dir
    for i in tqdm(range(len(images_list))):
        img = cv2.imread(images_list[i], 1)
        img = np.float32(img) / 255
        # Opencv loads as BGR:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        input_img = preprocess_image(img)

        grad_cam = GradCAM(model=model_ft, target_layers=target_layers, use_cuda=False)

        target_category = None

        grayscale_cam = grad_cam(input_img, target_category)
        grayscale_cam = grayscale_cam[0, :]
        cam = show_cam_on_image(img, grayscale_cam, use_rgb=True)

        row = i // 3
        col = (i % 3) * 2

        
        axes[row, col].imshow(img)
        axes[row, col].axis('off')

        axes[row, col + 1].imshow(cam)
        axes[row, col + 1].axis('off')

    plt.tight_layout()
    plt.show()

### **Plotting GradCam**

In [ ]:
fig, ax = plt.subplots(1,2)
img = cv2.imread(IMAGE_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cam = cv2.imread("./cam.jpg")
cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)
ax[0].imshow(img)
ax[1].imshow(cam)